# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770882981348                   -0.52    9.0    211ms
  2   -2.772148367335       -2.90       -1.32    1.0    108ms
  3   -2.772170227048       -4.66       -2.43    1.0    127ms
  4   -2.772170655877       -6.37       -3.14    1.0    110ms
  5   -2.772170722765       -7.17       -4.37    2.0    138ms
  6   -2.772170722898       -9.88       -4.58    1.0    110ms
  7   -2.772170723009       -9.95       -5.25    1.0    150ms
  8   -2.772170723014      -11.30       -5.68    1.0    118ms
  9   -2.772170723015      -12.09       -6.56    2.0    453ms
 10   -2.772170723015   +  -14.88       -6.71    1.0    412ms
 11   -2.772170723015   +  -14.65       -7.47    1.0    132ms
 12   -2.772170723015      -14.35       -8.25    2.0    134ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735580858634685

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770812365020                   -0.52    9.0    156ms
  2   -2.772062245286       -2.90       -1.32    1.0    118ms
  3   -2.772083066616       -4.68       -2.45    1.0    108ms
  4   -2.772083342710       -6.56       -3.15    1.0    124ms
  5   -2.772083417320       -7.13       -4.14    2.0    133ms
  6   -2.772083417688       -9.43       -4.57    1.0    113ms
  7   -2.772083417809       -9.92       -5.71    1.0    121ms
  8   -2.772083417810      -11.82       -5.75    2.0    127ms
  9   -2.772083417811      -12.37       -6.48    1.0    127ms
 10   -2.772083417811      -13.66       -7.47    1.0    130ms
 11   -2.772083417811      -14.27       -7.66    2.0    138ms
 12   -2.772083417811   +  -14.45       -8.66    1.0    130ms

Polarizability via ForwardDiff:       1.7725349768726653
Polarizability via finite difference: 1.7735580858634685
